<a href="https://colab.research.google.com/github/anjha1/Azure/blob/main/ML/Universal_Regression_Script_(Azure_ML_Ready).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ **1. Set up folder & copy dataset**

In [ ]:
import os, shutil

training_folder = 'linearreg-training-params'
os.makedirs(training_folder, exist_ok=True)

# Copy dataset to training folder
shutil.copy('data/your-dataset.csv', os.path.join(training_folder, "regression.csv"))


✅ **2. Training Script (linearreg_training.py)**

In [ ]:
%%writefile $training_folder/linearreg_training.py
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

run = Run.get_context()

# Accept CLI parameters
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, default=0.01)
parser.add_argument('--reg_type', type=str, default='l2')  # 'l1' for Lasso, 'l2' for Ridge
args = parser.parse_args()
reg = args.reg_rate
reg_type = args.reg_type

# Load dataset (adjust filename & columns as needed)
df = pd.read_csv('regression.csv')

# HARD-CODED feature & target columns
X = df[['feature1', 'feature2', 'feature3']].values
y = df['target_column'].values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose model
if reg_type == 'l1':
    model = Lasso(alpha=reg)
    run.log("Regularization Type", "L1 (Lasso)")
else:
    model = Ridge(alpha=reg)
    run.log("Regularization Type", "L2 (Ridge)")

# Train
run.log("Regularization Rate", np.float(reg))
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2 Score:", r2)

run.log("MSE", np.float(mse))
run.log("R2_Score", np.float(r2))

# Save model
os.makedirs('outputs', exist_ok=True)
joblib.dump(model, 'outputs/linear_model.pkl')

run.complete()


✅ **3. Run Training Using SKLearn Estimator**

In [ ]:
from azureml.core import Experiment
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Define estimator
estimator = SKLearn(
    source_directory=training_folder,
    entry_script='linearreg_training.py',
    script_params={
        '--reg_rate': 0.1,
        '--reg_type': 'l1'  # or 'l2'
    },
    compute_target='local'
)

experiment = Experiment(workspace=ws, name='linearreg-training')
run = experiment.submit(config=estimator)
RunDetails(run).show()
run.wait_for_completion()


✅ **4. Register the Trained Model**

In [ ]:
from azureml.core import Model

# Model metrics
model_props = {
    'MSE': run.get_metrics()['MSE'],
    'R2_Score': run.get_metrics()['R2_Score']
}

# Register model
run.register_model(
    model_path='outputs/linear_model.pkl',
    model_name='linear_regression_model',
    tags={'Framework': 'SKLearn', 'Type': 'L1' if estimator.script_params['--reg_type'] == 'l1' else 'L2'},
    properties=model_props
)

# Print registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag in model.tags:
        print(f'\t{tag}: {model.tags[tag]}')
    for prop in model.properties:
        print(f'\t{prop}: {model.properties[prop]}')
    print()
